In [1]:
# Enter the Matrix https://www.cell.com/action/showPdf?pii=S0168-9525%2818%2930124-0
# NMF 2007 Paper https://academic.oup.com/bioinformatics/article/23/12/1495/225472
# DeconRNASeq
%matplotlib inline

from collections import defaultdict, Counter
import urllib
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
import scipy.cluster.hierarchy as sch

import nimfa

In [2]:
#!pip install nimfa
import os
print(os.getcwd())

/home/vaid-computer/Neeraj/NMF_SP


In [3]:
organ = "PanCancer"
#gene_dataset = pd.read_csv('/Users/apple/Documents/RussWork/Survival_Prediction/Data/METABRIC_TCGA_PAM50.csv')
rdir = './Data/TCGA_' + organ + '_RNA-seq.tsv'
#gene_dataset = pd.read_csv(rdir)#, sep = " ",header = None)

gene_dataset = pd.read_csv(rdir, sep ='\t',low_memory=False)
#, sep = " ",header = None)

# Drop Not Available (NA) rows
gene_dataset = gene_dataset.dropna()


In [4]:
#Remove Patient ID or Entrez ID column
#gene_dataset = gene_dataset.drop(columns="Patient ID")
gene_dataset = gene_dataset.drop(columns="gene_id")
#gene_dataset = gene_dataset.drop(columns="Entrez_Gene_Id")
#gene_dataset = gene_dataset.drop(columns="ID")

gene_dataset.head(5) #[1:]

,TCGA-OR-A5J1-01A-11R-A29S-07,TCGA-OR-A5J2-01A-11R-A29S-07,TCGA-OR-A5J3-01A-11R-A29S-07,TCGA-OR-A5J5-01A-11R-A29S-07,TCGA-OR-A5J6-01A-31R-A29S-07,TCGA-OR-A5J7-01A-11R-A29S-07,TCGA-OR-A5J8-01A-11R-A29S-07,TCGA-OR-A5J9-01A-11R-A29S-07,TCGA-OR-A5JA-01A-11R-A29S-07,TCGA-OR-A5JB-01A-11R-A29S-07,...,TCGA-CG-4449-01A-01R-1157-13,TCGA-CG-4462-01A-01R-1157-13,TCGA-CG-4465-01A-01R-1157-13,TCGA-CG-4466-01A-01R-1157-13,TCGA-CG-4469-01A-01R-1157-13,TCGA-CG-4472-01A-01R-1157-13,TCGA-CG-4474-01A-02R-1157-13,TCGA-CG-4475-01A-01R-1157-13,TCGA-CG-4476-01A-01R-1157-13,TCGA-CG-4477-01A-01R-1157-13
1,3.2661,2.6815,1.7301,0.0000,0.0000,1.1673,1.4422,0.0000,4.4556,7.1293,...,4.358154,5.676995,5.219350,14.846708,20.115492,6.997533,18.311906,12.057112,18.628740,17.874417
2,3.9385,8.9948,6.5650,1.5492,4.4709,6.0529,2.2876,1.3599,5.0581,0.0000,...,2.656360,3.342794,2.423442,5.055287,11.626054,13.654193,7.417109,11.585177,11.482418,14.919338
3,149.1350,81.0777,86.4879,53.9117,66.9063,103.5060,94.9316,78.1955,69.2389,155.7090,...,633.299781,294.018042,686.569179,563.573453,1039.307597,639.238135,742.479964,506.336449,712.452165,703.713324
4,2034.1000,1304.9300,1054.6600,2350.8900,1257.9900,1866.4300,995.0270,1762.1200,1213.5300,2005.5700,...,1202.538277,644.002317,1181.884532,663.885074,647.530395,1297.152549,1152.909807,1375.495774,971.893874,1736.988111
6,274.2550,199.3020,348.3930,439.1940,149.2150,64.5808,377.9530,274.3640,243.1290,93.5725,...,87.408466,140.001693,108.852744,117.607340,303.513122,57.924566,166.813851,115.805483,188.696451,96.991756


In [ ]:
names = list(gene_dataset.columns)
os.mkdir('./Factors/' + organ)

sdir = './Factors/' + organ + '/TCGA_' + organ +'_names.txt'
np.savetxt(sdir, names, fmt='%s')

In [5]:
gene_dataset = np.array(gene_dataset.values)
# log2(x+1) transform
#gene_dataset = np.log2(gene_dataset+1)

In [6]:
gene_dataset = gene_dataset.transpose()
gene_dataset.shape
#np.sum(gene_dataset, axis = 1)

(11069, 16335)

In [7]:
# normalize data
#gene_dataset = preprocessing.Normalizer().fit_transform(gene_dataset.transpose())

min_max_scaler = preprocessing.MinMaxScaler()
gene_dataset = gene_dataset.transpose()
gene_dataset = min_max_scaler.fit_transform(gene_dataset)
gene_dataset = gene_dataset.transpose()

# level of sparsity
print(np.sum(gene_dataset != 0)/gene_dataset.size)

# negative elements
print(np.sum(gene_dataset < 0))

# remove negative elements
gene_dataset = gene_dataset - np.min(gene_dataset)


gene_dataset.shape

0.9813838746369401
0


(11069, 16335)

In [8]:
#np.min(gene_dataset, axis = 1)
gene_dataset.shape
#gene_dataset = gene_dataset.transpose()

(11069, 16335)

In [9]:
gene_dataset

array([[7.51509992e-06, 9.06225193e-06, 3.43150677e-04, ...,
        4.23607644e-03, 2.66343001e-03, 1.37150286e-03],
       [6.13903547e-06, 2.05927266e-05, 1.85619570e-04, ...,
        7.00555180e-03, 4.34068613e-03, 1.83527055e-03],
       [5.90828683e-06, 2.24194573e-05, 2.95355945e-04, ...,
        9.06890099e-03, 5.06256275e-03, 1.49327246e-03],
       ...,
       [9.14500372e-05, 8.80983945e-05, 3.60177888e-03, ...,
        2.20473853e-02, 9.73644584e-03, 3.37108426e-03],
       [4.01314402e-05, 2.54099702e-05, 1.46941217e-03, ...,
        6.80506967e-03, 3.94765124e-03, 2.11797084e-03],
       [1.29945782e-04, 1.09415419e-04, 4.89480089e-03, ...,
        1.73592929e-02, 8.67018629e-03, 6.37074157e-03]])

In [ ]:
# Find the exact rank
#ranks = [10,20,30,40,50,60,70,80,90, 100,110,120,130,140,150,160,170,180,190,200]
ranks = list(range(2,201))
metrics = {'rss':[],'evar':[],'cophenetic':[],'dispersion':[]}

for counter,item in enumerate(ranks):
    nmf = nimfa.Nmf(gene_dataset, rank=item, seed="random_vcol", max_iter=100, update='divergence',
                objective='div')
    nmf_fit = nmf()
    
    # Compute Metrics
    rss = nmf_fit.fit.rss()
    evar = nmf_fit.fit.evar()
    cophenetic = nmf_fit.fit.coph_cor()
    dispersion = nmf_fit.fit.dispersion()
    
    
    metrics['rss'].append(rss)
    metrics['evar'].append(evar)
    metrics['cophenetic'].append(cophenetic)
    metrics['dispersion'].append(dispersion)
    
    

    print(f"For rank:{item}, the metrics are - RSS: {rss}, Explained variance: {evar}, Cophenetic Correlation: {cophenetic}, Dispersion:{dispersion} \n")

For rank:2, the metrics are - RSS: 33825.08265050931, Explained variance: 0.5850019811947604, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:3, the metrics are - RSS: 32426.593407875804, Explained variance: 0.6021599663210615, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:4, the metrics are - RSS: 30562.224703630804, Explained variance: 0.6250338001141185, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:5, the metrics are - RSS: 29094.669879478253, Explained variance: 0.6430391469392566, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:6, the metrics are - RSS: 27965.4053807313, Explained variance: 0.6568940289665743, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:7, the metrics are - RSS: 26652.109100196314, Explained variance: 0.6730067864772526, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:8, the metrics are - RSS: 26097.10974072428, Explained variance: 0.6798160421115652, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:9, the m

For rank:60, the metrics are - RSS: 14563.22044571168, Explained variance: 0.8213246750986636, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:61, the metrics are - RSS: 15124.767784823598, Explained variance: 0.814435082673875, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:62, the metrics are - RSS: 15181.033093965343, Explained variance: 0.8137447667901697, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:63, the metrics are - RSS: 13928.84248162803, Explained variance: 0.8291078223266706, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:64, the metrics are - RSS: 14457.312555456483, Explained variance: 0.822624052991864, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:65, the metrics are - RSS: 14675.856812955391, Explained variance: 0.8199427459032635, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:66, the metrics are - RSS: 14181.126932003339, Explained variance: 0.8260125587270841, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:6

For rank:118, the metrics are - RSS: 11864.921281259476, Explained variance: 0.8544299543661671, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:119, the metrics are - RSS: 11742.44785102781, Explained variance: 0.8559325739289211, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:120, the metrics are - RSS: 12112.238842419443, Explained variance: 0.8513956292483958, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:121, the metrics are - RSS: 11857.223031534439, Explained variance: 0.8545244037550177, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:122, the metrics are - RSS: 12295.351927366448, Explained variance: 0.8491490252044238, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:123, the metrics are - RSS: 11749.301391837893, Explained variance: 0.8558484882257944, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:124, the metrics are - RSS: 11742.885086029533, Explained variance: 0.8559272095175066, Cophenetic Correlation: 1.0, Dispersion:1.0 



For rank:176, the metrics are - RSS: 11212.459527798788, Explained variance: 0.8624349705794327, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:177, the metrics are - RSS: 10646.487163218126, Explained variance: 0.8693788533904909, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:178, the metrics are - RSS: 10874.758491620245, Explained variance: 0.8665782054211801, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:179, the metrics are - RSS: 11419.578995865519, Explained variance: 0.8598938335837958, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:180, the metrics are - RSS: 10922.871425910269, Explained variance: 0.8659879105617242, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:181, the metrics are - RSS: 10558.764889639955, Explained variance: 0.8704551129850697, Cophenetic Correlation: 1.0, Dispersion:1.0 

For rank:182, the metrics are - RSS: 10437.609900908368, Explained variance: 0.8719415566639065, Cophenetic Correlation: 1.0, Dispersion:1.0 


In [ ]:
# Plotting the metrics
rss = metrics['rss']
# scale the rss value to [0,1]
max_rss = np.max(rss)
min_rss = np.min(rss)

rss = [(x-min_rss)/(max_rss-min_rss) for x in rss]
coph = metrics['cophenetic'] 
disp = metrics['dispersion']
evar = metrics['evar']
evar = [(x-np.min(evar))/(np.max(evar)-np.min(evar)) for x in evar]

plt.figure(figsize=(20,10))
plt.plot(ranks[:len(rss)], rss, 'o-', label='RSS', linewidth=2)
plt.plot(ranks[:len(rss)], coph, 'o-', label='Cophenetic correlation', linewidth=2)
plt.plot(ranks[:len(rss)], disp,'o-', label='Dispersion', linewidth=2)
plt.plot(ranks[:len(rss)], evar, 'o-', label='Explained variance', linewidth=2)
plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05), ncol=3, numpoints=1)
plt.savefig('./Factors/'+organ+'/nmf_10_to_100_'+organ+'.png');

In [ ]:
# Save the rank selection results
import pickle
with open('./Factors/'+organ+'/nmf_2_to_100_'+organ, 'wb') as f:
    pickle.dump(metrics, f)

# rank = 51
nmf = nimfa.Nmf(gene_dataset, rank=rank, seed='random_vcol', max_iter=1500, 
                update='divergence', objective='div', n_run=10, track_factor=True)
nmf_fit = nmf()

In [ ]:
gs = nmf_fit.fit.score_features()
gss = nmf_fit.fit.select_features()

In [ ]:
nmf_fit.fit.coef().shape

In [16]:
rr =99
nmf = nimfa.Nmf(gene_dataset, rank=rr, seed="random_vcol", max_iter=1500, update='divergence',
                objective='div')
nmf_fit = nmf()

In [15]:
nmf_fit.fit.coph_cor()

1.0

In [18]:
nmf_fit.fit.rss()

10830.016427389402

In [ ]:

nmf_fit.fit.coef()

In [17]:
import pickle
#rank = 89
sdir = './Factors/'+organ+'/nmf_rank'+str(rr) + '_' + organ
with open(sdir, 'wb') as f:
    pickle.dump(nmf_fit, f)

In [13]:
nmf_fit.fit.basis().shape

(11069, 100)

In [ ]:
gene_dataset.shape

In [14]:
sfile = sdir + '.csv'
np.savetxt(sfile, nmf_fit.fit.basis(), delimiter=",")

In [ ]:
gene_dataset.shape


In [ ]:
C = 1 - nmf_fit.fit.consensus()

In [ ]:
C.shape

In [ ]:
import matplotlib.gridspec as gridspec
import scipy.cluster.hierarchy as sch

def clean_axis(ax):
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    for sp in ax.spines.values():
        sp.set_visible(False)

fig = plt.figure(figsize=(13.9, 10))
heatmapGS = gridspec.GridSpec(1, 2, wspace=.01, hspace=0., width_ratios=[0.25,1])

C = 1 - nmf_fit.fit.consensus()
Y = sch.linkage(C, method='average')

denAX = fig.add_subplot(heatmapGS[0,0])
denD = sch.dendrogram(Y, orientation='right', link_color_func=lambda k: 'black')
clean_axis(denAX)

heatmapAX = fig.add_subplot(heatmapGS[0,1])
D = C[denD['leaves'], :][:, denD['leaves']]
axi = heatmapAX.imshow(D, interpolation='nearest', aspect='equal', origin='lower', cmap='RdBu') 
clean_axis(heatmapAX)

cb = fig.colorbar(axi, fraction=0.046, pad=0.04, aspect=10) 
cb.set_label('Distance', fontsize=20)